#### Dependencies

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Load Data

In [0]:
news = pd.read_csv("/content/drive/My Drive/News Virality Prediction/data/news_dataset.csv")

#### Cleaning

In [0]:
shares = news["Shares"]
for i in range(len(shares)):
  temp = shares[i]
  temp = temp.split(" ")
  temp = temp[0]
  if temp[-1] == "K":
    temp = float(temp[:-1]) * 1000
  elif temp[-1] == "M":
    temp = float(temp[:-1]) * 1000000
  else:
    temp = float(temp)
  shares[i] = temp
news["Shares"] = shares
news["Shares"] = news["Shares"].apply(lambda x: 1 if x>1500 else 0)

In [4]:
import sys
articles=news["Article"]
for i in range(len(articles)):
  if type(articles[i])==float:
    articles[i]="Nothing"

for k in range(1864):
  articles[k]=''.join([j for j in articles[k] if not j.isdigit()])
news["Article"]=articles

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
vocab_size=8000
embedding_dim=32
max_length=400
trunc_type='post'
oov_tok="<00V>"

In [6]:
for i in range(len(news)):
    news["Article"][i]=re.sub("[^a-zA-Z]"," ",news["Article"][i])
    news["Article"][i]=news["Article"][i].lower()
    news["Article"][i]=news["Article"][i].split()
    ps=PorterStemmer()
    news["Article"][i]=[ps.stem(word) for word in news["Article"][i] if not word in set(stopwords.words("english"))]
    news["Article"][i]=" ".join(news["Article"][i])

tokenizer = Tokenizer(num_words=vocab_size,oov_token="<OOV>")
tokenizer.fit_on_texts((news["Article"][:]).astype(str))
word_index = tokenizer.word_index
sequences=tokenizer.texts_to_sequences((news["Article"][:]).astype(str))
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type,padding="post")
X=padded
y=news.iloc[:,3].values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                            tf.keras.layers.GlobalAveragePooling1D(),
                           tf.keras.layers.Dropout(0.3),
                            tf.keras.layers.Dense(16,activation='relu',kernel_regularizer=tf.keras.regularizers.l2()),
                           tf.keras.layers.Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.l2()),
                            tf.keras.layers.Dense(1,activation='sigmoid')])
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.RMSprop(),metrics=['accuracy'])
num_epochs=100
model.fit(X,y,epochs=num_epochs,validation_data=(X_test,y_test))

Epoch 1/100
59/59 [==============================] - 0s 8ms/step - loss: 0.9985 - accuracy: 0.6432 - val_loss: 0.9110 - val_accuracy: 0.6193
Epoch 2/100
59/59 [==============================] - 0s 5ms/step - loss: 0.8358 - accuracy: 0.6491 - val_loss: 0.7959 - val_accuracy: 0.6193
Epoch 3/100
59/59 [==============================] - 0s 5ms/step - loss: 0.7410 - accuracy: 0.6491 - val_loss: 0.7259 - val_accuracy: 0.6193
Epoch 4/100
59/59 [==============================] - 0s 5ms/step - loss: 0.6891 - accuracy: 0.6491 - val_loss: 0.6880 - val_accuracy: 0.6193
Epoch 5/100
59/59 [==============================] - 0s 5ms/step - loss: 0.6601 - accuracy: 0.6491 - val_loss: 0.6733 - val_accuracy: 0.6193
Epoch 6/100
59/59 [==============================] - 0s 5ms/step - loss: 0.6505 - accuracy: 0.6491 - val_loss: 0.6694 - val_accuracy: 0.6193
Epoch 7/100
59/59 [==============================] - 0s 5ms/step - loss: 0.6503 - accuracy: 0.6491 - val_loss: 0.6682 - val_accuracy: 0.6193
Epoch 8/100
5

In [8]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>=0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[226,   5],
       [  1, 141]])